In [17]:
import submitit

import torch
import random
import numpy as np

import pickle
import itertools
import argparse
import logging
import os
import pathlib
import time
import json
import math
import matplotlib.pyplot as plt
from torch.utils import data
from fvcore.common.config import CfgNode
from submitit.core.utils import FailedJobError
from itertools import cycle
from time import sleep
from multi_objective.main import main, get_config

In [26]:
executor = submitit.AutoExecutor(folder="tmp/submitit")
seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [19]:
executor.update_parameters(timeout_min=20, slurm_partition="testdlc_gpu-rtx2080", name='results', gpus_per_node=1)
executor.update_parameters(slurm_array_parallelism=100)

In [23]:
def percent_finished(jobs):
    if len(jobs):
        return sum(job.done() for job in jobs) / len(jobs)

def execute(config, seeds):
    cfg = config.clone()
    cfg.eval_every = 1   # early stopping
    
    cfgs = []
    for seed in seeds:
        cfg = cfg.clone()
        cfg.merge_from_list(['seed', seed])
        cfgs.append(cfg)
    
    tags = [f"result_{s :2d}" for s in seeds]
    
    # func, rank, world_size, cfg, tag
    return executor.map_array(main, cycle([0]), cycle([1]), cfgs, tags)

#### cosmos

In [24]:
jobs = execute(get_config('configs/multi_mnist/cosmos.yaml'), seeds)

[SlurmJob<job_id=6285833_0, task_id=0, state="UNKNOWN">,
 SlurmJob<job_id=6285833_1, task_id=0, state="UNKNOWN">,
 SlurmJob<job_id=6285833_2, task_id=0, state="UNKNOWN">,
 SlurmJob<job_id=6285833_3, task_id=0, state="UNKNOWN">,
 SlurmJob<job_id=6285833_4, task_id=0, state="UNKNOWN">]

In [ ]:
jobs = execute(get_config('configs/multi_fashion/cosmos.yaml'), seeds)

In [ ]:
jobs = execute(get_config('configs/multi_fashion_mnist/cosmos.yaml'), seeds)

#### uniform

In [25]:
jobs = execute(get_config('configs/multi_mnist/uniform.yaml'), seeds)